# extract auditory features from audio clips

In [ ]:
#use conda env cochdnn

In [9]:

from __future__ import division
from scipy.io import wavfile
import os
import glob

# make sure we are using the correct plotting display. 
import matplotlib 
matplotlib.use('agg')
import matplotlib.pyplot as plt
import numpy as np

import sys
if sys.version_info < (3,):
    from StringIO import StringIO as BytesIO
else:
    from io import BytesIO
import base64

import scipy
import pickle
import h5py
import argparse

import torch
from robustness.tools.audio_helpers import load_audio_wav_resample
#from analysis_scripts.default_paths import fMRI_DATA_PATH

import itertools

#import build_network

ModuleNotFoundError: No module named 'torch'

In [2]:
def preproc_sound_np(sound):
    sound = sound - np.mean(sound)
    sound = sound/np.sqrt(np.mean(sound**2))*0.1
    sound = np.expand_dims(sound, 0)
    sound = torch.from_numpy(sound).float().cuda()
    return sound

In [3]:
import importlib

# Choose the model that will be loaded
model_dir = '/om2/user/jsmentch/cochdnn/model_directories/resnet50_word_speaker_audioset'

build_network_spec = importlib.util.spec_from_file_location("build_network",
                        os.path.join(model_dir, 'build_network.py'))
build_network = importlib.util.module_from_spec(build_network_spec)
build_network_spec.loader.exec_module(build_network)

model, ds, all_layers = build_network.main(return_metamer_layers=True)

=> loading checkpoint '/net/vast-storage.ib.cluster/scratch/vast/gablab/jsmentch/cochdnn/model_checkpoints/audio_rep_training_cochleagram_1/standard_training_word_and_audioset_and_speaker_decay_lr/542752d7-9849-49ff-b84a-6758a81585b4/5_checkpoint.pt'
=> loaded checkpoint '/net/vast-storage.ib.cluster/scratch/vast/gablab/jsmentch/cochdnn/model_checkpoints/audio_rep_training_cochleagram_1/standard_training_word_and_audioset_and_speaker_decay_lr/542752d7-9849-49ff-b84a-6758a81585b4/5_checkpoint.pt' (epoch 6)


In [16]:
##############Begin Define Parameters#################
stim='TP'
for stim in ['friends_s01e01a','friends_s01e01b','friends_s01e02a','friends_s01e02b']:

    save_features_dir = f'../data/{stim}_clips_cochresnet50/'
    input_dir = f'../data/{stim}_clips/'
    
    
    if not os.path.isdir(save_features_dir):
        os.mkdir(save_features_dir) 
    
    
    
    #############LOAD_AUDIO################
    # contains the metatdata for the list of presented sounds (should be in the correct order)
    #sound_list = np.load(os.path.join(fMRI_DATA_PATH, 'neural_stim_meta.npy'))
    sound_list = glob.glob(f'{input_dir}*.wav')
    sound_list_n=len(sound_list)
    #wavs_location = os.path.join(fMRI_DATA_PATH, '165_natural_sounds')
    
    SR=20000 # Match with the networks we are building/training
    MEASURE_DUR=2
    wav_array = np.empty([sound_list_n, SR*MEASURE_DUR])
    for wav_idx, wav_data in enumerate(sound_list):
        test_audio, SR = load_audio_wav_resample(wav_data, DUR_SECS=MEASURE_DUR, resample_SR=SR)
        wav_array[wav_idx,:] = test_audio/np.sqrt(np.mean(test_audio**2))
    
    # Measure the activations for each sound for each layer, and put the input in the dictionary array. 

    filename = 'cochresnet50_activations'
    # only use the non-fake layers
    all_layers = [e.split('_fake')[0] for e in all_layers] # Don't duplicate these since we aren't synthesizing
    new_all_layers = []
    for l_unique in all_layers:
        if l_unique not in new_all_layers:
            new_all_layers.append(l_unique)
    all_layers = new_all_layers
    net_layer_dict = {}
    net_layer_dict_full = {}
    net_h5py_file = h5py.File(os.path.join(save_features_dir, filename + '.h5'), "w")
    net_h5py_file_full = h5py.File(os.path.join(save_features_dir, filename + '_full.h5'), "w")
    
    # Save the list of layers to the hdf5
    net_h5py_file['layer_list'] = np.array([layer.encode("utf-8") for layer in all_layers])
    net_h5py_file_full['layer_list'] = np.array([layer.encode("utf-8") for layer in all_layers])
    
    for sound_idx, sound_info in enumerate(sound_list):
        sound = preproc_sound_np(wav_array[sound_idx,:])
        with torch.no_grad():
            (predictions, rep, layer_returns), orig_image = model(sound, with_latent=True) # Corresponding representation
    
        # Make the array have the correct size
        if sound_idx == 0:
            for layer in all_layers:
                print(layer)
                layer_shape_165 = layer_returns[layer].shape
                layer_shape_full = np.prod(np.array(layer_shape_165))
                if len(layer_shape_165)==4:
                    layer_shape_unraveled = layer_shape_165[1]*layer_shape_165[2]# don't take the time dimension into account
                else:
                    layer_shape_unraveled = layer_shape_165[1]
                net_layer_dict_full[layer] = net_h5py_file_full.create_dataset(layer, (sound_list_n, layer_shape_full), dtype='float32')
                net_layer_dict[layer] = net_h5py_file.create_dataset(layer, (sound_list_n, layer_shape_unraveled), dtype='float32')
    
        for layer_idx, layer in enumerate(all_layers):
            # time averaged features, so that they can be related to the fMRI activations
            if layer_returns[layer].ndim==4: # NCHW (W is time)
                net_layer_dict[layer][sound_idx,:] = np.mean(layer_returns[layer].cpu().detach().numpy(),3).ravel()
            else: # fully connected layers do not have a temporal component.  
                net_layer_dict[layer][sound_idx,:] = layer_returns[layer].cpu().detach().numpy().ravel()
            net_layer_dict_full[layer][sound_idx,:] = layer_returns[layer].cpu().detach().numpy().ravel()
    net_h5py_file.close()
    net_h5py_file_full.close()

/tmp/ipykernel_40925/6305016.py:26: RuntimeWarning: invalid value encountered in divide
  wav_array[wav_idx,:] = test_audio/np.sqrt(np.mean(test_audio**2))
/tmp/ipykernel_40925/6305016.py:26: RuntimeWarning: invalid value encountered in sqrt
  wav_array[wav_idx,:] = test_audio/np.sqrt(np.mean(test_audio**2))


input_after_preproc
conv1
bn1
conv1_relu1
maxpool1
layer1
layer2
layer3
layer4
avgpool
final/signal/word_int
final/signal/speaker_int
final/noise/labels_binary_via_int
input_after_preproc
conv1
bn1
conv1_relu1
maxpool1
layer1
layer2
layer3
layer4
avgpool
final/signal/word_int
final/signal/speaker_int
final/noise/labels_binary_via_int
input_after_preproc
conv1
bn1
conv1_relu1
maxpool1
layer1
layer2
layer3
layer4
avgpool
final/signal/word_int
final/signal/speaker_int
final/noise/labels_binary_via_int
input_after_preproc
conv1
bn1
conv1_relu1
maxpool1
layer1
layer2
layer3
layer4
avgpool
final/signal/word_int
final/signal/speaker_int
final/noise/labels_binary_via_int


In [14]:
all_layers

['input_after_preproc',
 'conv1',
 'bn1',
 'conv1_relu1',
 'maxpool1',
 'layer1',
 'layer2',
 'layer3',
 'layer4',
 'avgpool',
 'final/signal/word_int',
 'final/signal/speaker_int',
 'final/noise/labels_binary_via_int']

input_after_preproc
conv1
bn1
conv1_relu1
maxpool1
layer1
layer2
layer3
layer4
avgpool
final/signal/word_int
final/signal/speaker_int
final/noise/labels_binary_via_int


In [ ]:
'''
Ian: Another slight detail. You’ll notice there are two h5 files being made to save the layer embeddings:
net_h5py_file_full and net_h5py_file. For different reasons, Jenelle would often work with time-averaged 
representations but saved the time-averaged and full ones separately just in case (to avoid having to
average every time). Full is the full embedding and the other is the time averaged one.  If you’re ever
confused in this repo, representations, activations, and embeddings all mean the same thing here
'''

## look at the extracted features

In [7]:
all_layers=['input_after_preproc',
 'conv1',
 'bn1',
 'conv1_relu1',
 'maxpool1',
 'layer1',
 'layer2',
 'layer3',
 'layer4',
 'avgpool',
 'final/signal/word_int',
 'final/signal/speaker_int',
 'final/noise/labels_binary_via_int']

In [15]:
import os
import numpy as np
import glob

import h5py


stim='DM'
save_features_dir = f'../data/{stim}_clips_cochresnet50/'

print('CochResNet50 time-averaged')
# Open the file 'myfile.h5' in read-only mode
file = h5py.File(f'{save_features_dir}cochresnet50_activations.h5', 'r')
for layer in all_layers:
# # Now you can access datasets within the file
    data = file[layer]
    print(data.shape, layer)
# # Don't forget to close the file when you're done
file.close()

print('CochResNet50 full')
# Open the file 'myfile.h5' in read-only mode
file = h5py.File(f'{save_features_dir}cochresnet50_activations_full.h5', 'r')
for layer in all_layers:
# # Now you can access datasets within the file
    data = file[layer]
    print(data.shape, layer)
# # Don't forget to close the file when you're done
file.close()

CochResNet50 time-averaged
(749, 211) input_after_preproc
(749, 6784) conv1
(749, 6784) bn1
(749, 6784) conv1_relu1
(749, 3392) maxpool1
(749, 13568) layer1
(749, 13824) layer2
(749, 14336) layer3
(749, 14336) layer4
(749, 2048) avgpool
(749, 794) final/signal/word_int
(749, 433) final/signal/speaker_int
(749, 517) final/noise/labels_binary_via_int
CochResNet50 full
(749, 82290) input_after_preproc
(749, 1322880) conv1
(749, 1322880) bn1
(749, 1322880) conv1_relu1
(749, 332416) maxpool1
(749, 1329664) layer1
(749, 677376) layer2
(749, 358400) layer3
(749, 186368) layer4
(749, 2048) avgpool
(749, 794) final/signal/word_int
(749, 433) final/signal/speaker_int
(749, 517) final/noise/labels_binary_via_int


In [11]:
data[0].shape

(82290,)

In [ ]:
CochResNet50 time-averaged
(749, 211) input_after_preproc (cochleagram)
(749, 6784) conv1
(749, 6784) bn1
(749, 6784) conv1_relu1
(749, 3392) maxpool1
(749, 13568) layer1
(749, 13824) layer2
(749, 14336) layer3
(749, 14336) layer4
(749, 2048) avgpool
(749, 794) final/signal/word_int
(749, 433) final/signal/speaker_int
(749, 517) final/noise/labels_binary_via_int

## pyloudnorm

In [2]:
import librosa
import pyloudnorm as pyln
import glob
import numpy as np


In [8]:
stim='DM'
stim='TP'

# Load and normalize the audio data
data, rate = librosa.load(f'../data/{stim}.wav', sr=20000)
data = data / np.max(np.abs(data))  # Normalizing audio

# Define the window size (800 ms) in terms of samples
window_size = int(0.800 * rate)

# Create a loudness meter
meter = pyln.Meter(rate)

# Initialize lists to store loudness and RMS values
loudness_values = []
rms_values = []

# Loop through the data in 800 ms chunks
for i in range(0, len(data), window_size):
    chunk = data[i:i + window_size]
    
    if len(chunk) < window_size:
        break
    
    # Calculate RMS
    rms = np.sqrt(np.mean(chunk**2))
    
    # Store the RMS value
    rms_values.append(rms)
    
    # Only calculate loudness if RMS exceeds the threshold
    loudness = meter.integrated_loudness(chunk)
    
    
    # Store the loudness value
    loudness_values.append(loudness)

# Convert lists to numpy arrays for further processing
loudness_array = np.array(loudness_values)
rms_array = np.array(rms_values)

# Print or compare the loudness and RMS arrays
# print("Loudness (LUFS):", loudness_array)
# print("RMS:", rms_array)
np.save(f'../data/features/{stim}_lla_rms', rms_array)
np.save(f'../data/features/{stim}_lla_lufs', loudness_values)